# Ejercicio 1

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style()

In [3]:
df = pd.read_csv('compresive_strength_concrete.csv')

In [4]:
df.describe()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


# Ejercicio 2

In [5]:
columnas_antiguas = df.columns
columnas_nuevas = columnas_antiguas.str.split('(').str[0].str.strip()
df.columns = columnas_nuevas

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Cement                         1030 non-null   float64
 1   Blast Furnace Slag             1030 non-null   float64
 2   Fly Ash                        1030 non-null   float64
 3   Water                          1030 non-null   float64
 4   Superplasticizer               1030 non-null   float64
 5   Coarse Aggregate               1030 non-null   float64
 6   Fine Aggregate                 1030 non-null   float64
 7   Age                            1030 non-null   int64  
 8   Concrete compressive strength  1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


- La descripcion de las variables (desde Cement hasta Age) de la base de datos muestra que todas corresponden a valores decimales, exceptuando Age. Ademas, ninguna presenta valores faltantes.

- La descripcion del vector objetivo (Concrete compressive strengt) indica que es una variable de valores decimales - continua. Tampoco presenta valores faltantes.

# Ejercicio 3

- Debido a que el vector objetivo es una variable continua, el modelo a utilizar debe ser de regresion (y no de clasificacion). Como se utilizara un modelo aditivo general, se importara LinearGAM de la libreria pygam.

- Como se hara una prediccion de una variable continua, las metricas que se utilizaran en el modelo seran r2_score y mean_squared_error

- Para construir el modelo, se utilizaran las siguientes librerias:

    - sklearn.preprocessing para StandardScaler
    - sklearn.model_selection para train_test_split

In [7]:
from pygam import LinearGAM, s

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [8]:
X = df.drop(columns='Concrete compressive strength')
y = df['Concrete compressive strength']

In [9]:
X_train_pre, X_test_pre, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [10]:
scaler = StandardScaler().fit(X_train_pre)

In [11]:
X_train = pd.DataFrame(scaler.transform(X_train_pre), columns=X_train_pre.columns)
X_test = pd.DataFrame(scaler.transform(X_test_pre), columns=X_test_pre.columns)

In [22]:
gam = LinearGAM(s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7), fit_intercept=True)
gam.fit(X_train, y_train)

LinearGAM(callbacks=[Deviance(), Diffs()], fit_intercept=True, 
   max_iter=100, scale=None, 
   terms=s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + intercept,
   tol=0.0001, verbose=False)

In [26]:
lams = np.logspace(-3, 3, 3)
lams = [lams] * len(X_train.columns)

In [30]:
gam_2 = LinearGAM(s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7), fit_intercept=True)
gam_2.gridsearch(X_train.values, y_train.values, lam=lams)

  0% (0 of 6561) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--
  0% (1 of 6561) |                       | Elapsed Time: 0:00:00 ETA:   0:33:34
  0% (2 of 6561) |                       | Elapsed Time: 0:00:00 ETA:   0:24:02
  0% (3 of 6561) |                       | Elapsed Time: 0:00:00 ETA:   0:20:17
  0% (4 of 6561) |                       | Elapsed Time: 0:00:00 ETA:   0:18:14
  0% (5 of 6561) |                       | Elapsed Time: 0:00:00 ETA:   0:17:26
  0% (6 of 6561) |                       | Elapsed Time: 0:00:00 ETA:   0:16:41
  0% (7 of 6561) |                       | Elapsed Time: 0:00:01 ETA:   0:16:07
  0% (8 of 6561) |                       | Elapsed Time: 0:00:01 ETA:   0:15:40
  0% (9 of 6561) |                       | Elapsed Time: 0:00:01 ETA:   0:15:13
  0% (10 of 6561) |                      | Elapsed Time: 0:00:01 ETA:   0:14:56
  0% (11 of 6561) |                      | Elapsed Time: 0:00:01 ETA:   0:14:38
  0% (12 of 6561) |                     

LinearGAM(callbacks=[Deviance(), Diffs()], fit_intercept=True, 
   max_iter=100, scale=None, 
   terms=s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + intercept,
   tol=0.0001, verbose=False)

In [31]:
gam.summary()

LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     79.5201
Link Function:                     IdentityLink Log Likelihood:                                 -2908.3039
Number of Samples:                          690 AIC:                                              5977.648
                                                AICc:                                            5999.2231
                                                GCV:                                               33.4083
                                                Scale:                                             26.5582
                                                Pseudo R-Squared:                                   0.9178
Feature Function                  Lam

/tmp/ipykernel_8626/3358381670.py:1: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  gam.summary()
